In [33]:
!pip install docx2txt
!pip install python-docx
!pip install pdfplumber
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.5 MB/s eta 0:00:00


#Importing Libraries

In [2]:
import os
import re
import docx2txt
from docx import Document
from bs4 import BeautifulSoup
import pdfplumber
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

#Extracting Text from documents

In [3]:
def extract_text_from_resume(file_path):
    _, file_extension = os.path.splitext(file_path)

    if file_extension == '.txt':
        # Read plain text file
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

    elif file_extension == '.docx':
        # Extract text from DOCX file
        text = docx2txt.process(file_path)
        return text

    elif file_extension == '.doc':
        # Extract text from DOC file
        doc = Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + '\n'
        return text
    elif file_extension == '.html' or file_extension == '.htm':
        # Extract text from HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')
            text = soup.get_text(separator='\n')
            return text

    elif file_extension == '.pdf':
        # Extract text from PDF file using pdfplumber
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page_num in range(len(pdf.pages)):
                text += pdf.pages[page_num].extract_text()
            return text
    else:
        print("Unsupported file format")


In [4]:
#Checking for PDF File
file_path = '/content/Maaz khan _ d17 (1).pdf'
resume_text = extract_text_from_resume(file_path)
print(resume_text)

Maaz Muhammad Khan
maazkhan1211@gmail.com | (216)-666-095|
Education
M.S. Computer Science (GPA 3.77) January 2020 –December 2021
Cleveland State University
Relevant Coursework: Data Mining, Big data, Machine Learning, Artificial Intelligence, Data communication and Networks
B.S. Computer Science (GPA 3.39) January 2015 – December 2018
Muhammad Ali Jinnah University
Relevant Coursework: Database Systems, Cloud Computing, Statistics, Algorithms and Data Structures, Object Oriented Programming,
Human Computer Interaction, Web Development, Computer Architecture, Software Requirement Engineering, Software Quality Assurance
Work Experience
Business Intelligence Analyst\Data Engineer Jan 2022-Present
New York Air Brake, Watertown, NY
• Designed & developed visual BI reports and dashboards to support management decision-making using Power bi and Tableau.
• Developed and automated data pipelines and ETL process and deployed new data models to make datasets readily-consumable by visualization 

In [ ]:
#Checking For docx file
file_path = 'Praveen Batchu .docx'
resume_text = extract_text_from_resume(file_path)
print(resume_text)

Praveen V Batchu

Cell: 503-547-4720

SUMMARY



Over 15 years of experience in the IT industry, with a wide range of experience in Database Administration, System Analysis, Design, Development, Testing & Support of MS SQL Servers in Production, Development environments & Replication Environments.

Upgrading of Database from SQL Server 2000 to 2005 and 2008/R2 version.

Provided optimal database file layout for utilization of storage and performance acceptable to the client.

Developed & maintained scripts, utility jobs or implemented needed software for database maintenance.

Implemented & managed database clustering, failover and load balance technologies as per client requirement.

Developed, tested & implemented a robust database backup and recovery strategy with client assistance.

Database recoveries as per requirements.

Conducted root cause and analysis for database incidents and problems.

Participated in designing, testing and implementing an enterprise Disaster Recovery Plan

In [ ]:
#For HTML file
file_path = 'demo.html'
resume_text = extract_text_from_resume(file_path)
print(resume_text)











Resume










John Doe


Email: john.doe@example.com


Phone: 123-456-7890






Education




Bachelor of Science in Computer Science


University of Example, 2015 - 2019


GPA: 3.8








Experience




Software Developer Intern


XYZ Company, Summer 2018




Developed web applications using HTML, CSS, and JavaScript.


Assisted in debugging and testing of software components.










Skills






Programming Languages: Python, Java, C++


Web Development: HTML, CSS, JavaScript


Frameworks: Django, React


Database: SQL, MongoDB














#Classification through fine tune model

In [6]:
model = DistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Mined/fine_tuned_distilbert_final")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def process_resume_lines(lines, model, tokenizer, label_list):
    final = {label: [] for label in label_list}

    # Keep track of previous and next lines during iteration
    prev_label = None
    prev_line = None

    for i, line in enumerate(lines):
        # Tokenize input text
        inputs = tokenizer(line, return_tensors="pt")

        # Perform inference
        with torch.no_grad():
            outputs = model(**inputs)

        # Get predicted label
        predicted_label = torch.argmax(outputs.logits).item()

        # Convert label index back to original label
        predicted_label_text = label_list[predicted_label]

        # Check if the predicted label is the same as the previous and next lines
        if prev_label == predicted_label_text or (prev_line and predicted_label_text == prev_line):
            predicted_label_text = prev_label if prev_label else prev_line
        # print("prev",prev_label)
        # Check if the next line has the same predicted label
        if i < len(lines) - 1:
            next_line = lines[i + 1]
            inputs_next = tokenizer(next_line, return_tensors="pt")
            with torch.no_grad():
                outputs_next = model(**inputs_next)
            predicted_label_next = torch.argmax(outputs_next.logits).item()
            predicted_label_text_next = label_list[predicted_label_next]
            # print("next",predicted_label_text_next)
            if predicted_label_text_next == prev_label :
                predicted_label_text = predicted_label_text_next

        # Append the line to the final result
        final[predicted_label_text].append(line)
        # print("cur",predicted_label_text)


        # Update previous label and line for the next iteration
        prev_label = predicted_label_text
        prev_line = line

    return final

In [8]:
label_list = ["Education", "Experience", "Projects", "Skills", "Summary"]
# Process lines and update labels
with pdfplumber.open(file_path) as pdf:
    all_lines = []
    for page in pdf.pages:
        all_lines.extend(page.extract_text().splitlines())
final_result = process_resume_lines(all_lines, model, tokenizer, label_list)


In [9]:
for key, value in final_result.items():
    print(key)
    i=1
    for li in value:
      print(i,end=" ")
      i=i+1
      print(li)
    print("-----------------------------------------------------------------------------")

Education
1 Maaz Muhammad Khan
2 maazkhan1211@gmail.com | (216)-666-095|
3 Education
4 M.S. Computer Science (GPA 3.77) January 2020 –December 2021
5 Cleveland State University
6 Relevant Coursework: Data Mining, Big data, Machine Learning, Artificial Intelligence, Data communication and Networks
7 B.S. Computer Science (GPA 3.39) January 2015 – December 2018
8 Muhammad Ali Jinnah University
9 New York Air Brake, Watertown, NY
10 Research Assistant Feb 2020 – April 2021
11 Artificial Intelligence and Computer Vision Lab, Cleveland State University, Cleveland, OH.
12 John Carroll University, Cleveland, OH
-----------------------------------------------------------------------------
Experience
1 Business Intelligence Analyst\Data Engineer Jan 2022-Present
2 • Developed and automated data pipelines and ETL process and deployed new data models to make datasets readily-consumable by visualization tools
3 using Snowflake and SQL Server.
4 • Drove efficiencies throughout the organization by 

#Time Flow

In [10]:
def extract_career_timeline(resume_text):

  date_pattern = re.compile(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|[A-Za-z]+)\s(?:\d{4}|\d{1,2},\s\d{4})\b')
  job_pattern = re.compile(r'\b(?:[A-Z][a-z]+\s)+[A-Z][a-z]+\b(?!\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))')
  place_pattern = re.compile(r'(?:(?<=Present)|(?<=\d{4}))[^\d]*(?:\b(?:[A-Z][a-z]+\s?)+[A-Z][a-z]+\b)?')


  timeline = []

  for line in resume_text.split('\n'):
    dates = date_pattern.findall(line)
    jobs = job_pattern.findall(line)
    places = place_pattern.findall(line)

    for i in range(min(len(dates), len(jobs), len(places))):
      timeline.append((dates[i], jobs[i], places))
      timeline.sort(key=lambda x: convert_to_year(x[0]))

  return timeline

def convert_to_year(date):

    try:
        return int(re.search(r'\d{4}', date).group())
    except ValueError:
        return 0


career_timeline = extract_career_timeline(resume_text)


print(" -> ".join([f"{date} - {role}" for date, role, _ in career_timeline]))

January 2015 - Computer Science -> May 2016 - Enginer May -> May 2019 - Systems Administrator May -> January 2020 - Computer Science -> Feb 2020 - Research Assistant Feb -> Jan 2022 - Business Intelligence Analyst


# Classication

In [36]:
file_path = 'data.txt'
d={}
titles=[]
with open(file_path, 'r') as file:
    for line in file:
        # output_data=line.replace("-",":")
        # print(output_data)
        X,y=line.split('-')
        X=X.replace(":","-")
        X=X.strip().replace("/n","")

        y=y.strip().replace("/n","")
        if y in d:
            d[y].append(X.lower())  # Convert 'X' to lowercase before appending
        else:
            d[y] = [X.lower()]  # Convert 'X' to lowercase before adding to a new list
        titles.append(y.lower())


        d[y].append(X)

In [37]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

def preprocess_text(text):
    words = word_tokenize(text.lower())
    print(words)
    return set(words)

def find_similar_words(text1, text2):
    processed_text1 = preprocess_text(text1)
    processed_text2 = preprocess_text(text2)

    common_words = processed_text1.intersection(processed_text2)

    return common_words



text1 = ' '.join(titles)

text2=final_result['Experience']
text2 = '\n'.join(text2)
text2=text2.replace("\n"," ")

similar_words = find_similar_words(text1, text2)
if similar_words:
  print(f"Similar words between the two text files: {', '.join(similar_words)}")
else:
  print("No similar words found.")


['computer', 'software', 'engineers', 'computer', 'and', 'information', 'research', 'scientists', 'web', 'developers', 'network', 'and', 'computer', 'systems', 'administrators', 'information', 'security', 'analyst', 'database', 'administrators', 'computer', 'systems', 'analysts', 'computer', 'systems', 'analysts', 'computer', 'systems', 'analysts', 'computer', 'and', 'information', 'systems', 'managers', 'computer', 'user', 'support', 'specialists', 'software', 'developers', ',', 'applications', 'computer', 'network', 'architects', 'computer', 'and', 'information', 'research', 'scientists', 'software', 'quality', 'assurance', 'analysts', 'and', 'testers', 'database', 'architects', 'software', 'developers', ',', 'systems', 'software', 'information', 'security', 'managers', 'multimedia', 'artists', 'and', 'animators', 'web', 'developers', 'web', 'developers', 'technical', 'writers', 'software', 'developers', ',', 'systems', 'software', 'computer', 'systems', 'analysts', 'network', 'and',

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
import Levenshtein

def calculate_levenshtein_distance(str1, str2):
    return Levenshtein.distance(str1, str2)

def find_distance(input_string, job_dict):
    distances = {}
    for job_category, job_titles in job_dict.items():
        job_distances = []
        for job_title in job_titles:
            distance = calculate_levenshtein_distance(input_string, job_title)
            job_distances.append(distance)
        distances[job_category] = min(job_distances)

    top_distances = sorted(distances.items(), key=lambda x: x[1])[:3]
    return top_distances

allcomb = []
similar_words = list(similar_words)

for i in range(len(similar_words)):
    for j in range(len(similar_words)):
        if i != j:
            allcomb.append(' '.join([similar_words[i], similar_words[j]]))

top_combinations = []

for combination in allcomb:
    top_distances = find_distance(combination, d)
    min_distance = min(distance for category, distance in top_distances)
    min_distance_category = [category for category, distance in top_distances if distance == min_distance][0]
    top_combinations.append((combination, min_distance, min_distance_category))

top_combinations = sorted(top_combinations, key=lambda x: x[1])[:5]

for combination, distance, category in top_combinations:
    print(f"Combination: {combination}, Distance: {distance}, Category: {category}")


Combination: systems quality, Distance: 5, Category: Computer Systems Analysts
Combination: systems and, Distance: 5, Category: Computer Systems Analysts
Combination: systems managers, Distance: 5, Category: Computer Systems Analysts
Combination: systems ,, Distance: 7, Category: Computer Systems Analysts
Combination: and quality, Distance: 8, Category: Computer Systems Analysts
